# Zero Shot Learning 

In [9]:
ckpt = torch.load("test_feats.pt", map_location="cpu")
X_test, y = ckpt["X_test"], ckpt["y"]

/tmp/ipykernel_1366/2809525365.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("test_feats.pt", map_location="cpu")


In [5]:
emb = torch.load("emb.pt", map_location="cpu")

/tmp/ipykernel_1366/543991217.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load("emb.pt", map_location="cpu")


In [11]:
ckpt = torch.load("train_feats.pt", map_location="cpu")
X_train, y = ckpt["X_train"], ckpt["y"]

/tmp/ipykernel_1366/943266446.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("train_feats.pt", map_location="cpu")


In [6]:
import torch, torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.models as tvm
import numpy as np
import torchvision.models as models
from torchvision.models import ResNet50_Weights 



In [12]:
X_test_parts, y_parts = [], []

for i in range(0, 50, 5):  
    #for k = 10, we divide test set into 10 parts, and each parts have 5 unseen categories
    mask = (y >= i) & (y < i + 5)
    X_test_parts.append(X_test[mask])
    y_parts.append(y[mask])

(X_test_1, X_test_2, X_test_3, X_test_4, X_test_5,
 X_test_6, X_test_7, X_test_8, X_test_9, X_test_10) = X_test_parts

(y_test_1, y_test_2, y_test_3, y_test_4, y_test_5,
 y_test_6, y_test_7, y_test_8, y_test_9, y_test_10) = y_parts
# check
for i, x in enumerate(X_test_parts, 1):
    print(f"X_test_{i}.shape = {x.shape}, y range = {y_parts[i-1].min().item()}–{y_parts[i-1].max().item()}")



X_test_1.shape = torch.Size([2426, 2048]), y range = 0–4
X_test_2.shape = torch.Size([4556, 2048]), y range = 5–9
X_test_3.shape = torch.Size([4541, 2048]), y range = 10–14
X_test_4.shape = torch.Size([4833, 2048]), y range = 15–19
X_test_5.shape = torch.Size([4108, 2048]), y range = 20–24
X_test_6.shape = torch.Size([2728, 2048]), y range = 25–29
X_test_7.shape = torch.Size([3786, 2048]), y range = 30–34
X_test_8.shape = torch.Size([3594, 2048]), y range = 35–39
X_test_9.shape = torch.Size([3599, 2048]), y range = 40–44
X_test_10.shape = torch.Size([3123, 2048]), y range = 45–49


In [14]:
import torch

X_train_parts, y_train_parts = [], []

for i in range(0, 50, 5):
    # same process but need change mask to proof unseen for train sets
    mask = (y < i) | (y >= i + 5)
    X_train_parts.append(X_train[mask])
    y_train_parts.append(y[mask])


(X_train_1, X_train_2, X_train_3, X_train_4, X_train_5,
 X_train_6, X_train_7, X_train_8, X_train_9, X_train_10) = X_train_parts

(y_train_1, y_train_2, y_train_3, y_train_4, y_train_5,
 y_train_6, y_train_7, y_train_8, y_train_9, y_train_10) = y_train_parts

# === check each train split's class distribution ===
for i in range(10):
    y_i = y_train_parts[i]
    uniq, counts = torch.unique(y_i, return_counts=True)
    uniq, counts = uniq.tolist(), counts.tolist()
    
    print(f"\nX_train_{i+1}.shape = {X_train_parts[i].shape}")
    print("distribution（class: count）:")
    for j in range(0, len(uniq), 10):
        line = ", ".join(f"{c}:{n}" for c, n in zip(uniq[j:j+10], counts[j:j+10]))
        print("   " + line)




X_train_1.shape = torch.Size([34868, 2048])
distribution（class: count）:
   5:895, 6:567, 7:728, 8:1028, 9:1338, 10:549, 11:1344, 12:946, 13:1038, 14:664
   15:1033, 16:874, 17:1202, 18:872, 19:852, 20:779, 21:684, 22:1645, 23:709, 24:291
   25:720, 26:1019, 27:100, 28:704, 29:185, 30:730, 31:728, 32:747, 33:713, 34:868
   35:1088, 36:512, 37:310, 38:696, 39:988, 40:1420, 41:500, 42:188, 43:291, 44:1200
   45:877, 46:215, 47:272, 48:589, 49:1170

X_train_2.shape = torch.Size([32738, 2048])
distribution（class: count）:
   0:1046, 1:383, 2:193, 3:174, 4:630, 10:549, 11:1344, 12:946, 13:1038, 14:664
   15:1033, 16:874, 17:1202, 18:872, 19:852, 20:779, 21:684, 22:1645, 23:709, 24:291
   25:720, 26:1019, 27:100, 28:704, 29:185, 30:730, 31:728, 32:747, 33:713, 34:868
   35:1088, 36:512, 37:310, 38:696, 39:988, 40:1420, 41:500, 42:188, 43:291, 44:1200
   45:877, 46:215, 47:272, 48:589, 49:1170

X_train_3.shape = torch.Size([32753, 2048])
distribution（class: count）:
   0:1046, 1:383, 2:193, 3:1

In [15]:
print(emb.shape, 
X_train_1.shape ,
X_test_1.shape, 
y_train_1.shape ,
y_test_1.shape) 

torch.Size([50, 768]) torch.Size([34868, 2048]) torch.Size([2426, 2048]) torch.Size([34868]) torch.Size([2426])


In [ ]:

import functools, builtins
print = functools.partial(builtins.print, flush=True)

import torch, numpy as np
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
torch.manual_seed(42); torch.cuda.manual_seed_all(42)
print(">>> Script loaded. Device:", device)


def l2norm(X):
    return X / (X.norm(dim=1, keepdim=True) + 1e-12)

def unique_sorted_cpu(y_tensor):
    return sorted(list(set(y_tensor.detach().cpu().numpy().tolist())))

def split_by_val_classes(X, y, val_classes):
    s = set(val_classes)
    mask_val = torch.tensor([int(int(t.item()) in s) for t in y], device=y.device, dtype=torch.bool)
    mask_tr  = ~mask_val
    return X[mask_tr], y[mask_tr], X[mask_val], y[mask_val]

def accuracy_manual(y_true, y_pred):
    return sum(1 for a,b in zip(y_true, y_pred) if a == b) / len(y_true)

def confusion_matrix_manual(y_true, y_pred, labels):
    L = len(labels); idx = {c:i for i,c in enumerate(labels)}
    cm = torch.zeros((L, L), dtype=torch.int64)
    for t,p in zip(y_true, y_pred):
        cm[idx[t], idx[p]] += 1
    return cm

def macro_prf_from_cm(cm):
    cm = cm.to(torch.float64)
    tp = cm.diag(); fp = cm.sum(0) - tp; fn = cm.sum(1) - tp
    prec_i = torch.where(tp+fp>0, tp/(tp+fp), torch.zeros_like(tp))
    reca_i = torch.where(tp+fn>0, tp/(tp+fn), torch.zeros_like(tp))
    f1_i   = torch.where(prec_i+reca_i>0, 2*prec_i*reca_i/(prec_i+reca_i), torch.zeros_like(tp))
    return float(prec_i.mean()), float(reca_i.mean()), float(f1_i.mean())

def rmse_mae_visual(X_real, y_pred, gen_means_dict):
    diffs = []
    for xr, c in zip(X_real, y_pred):
        mu = gen_means_dict[int(c)]
        diffs.append((xr - mu).detach().cpu().numpy())
    diffs = np.stack(diffs, 0)
    mse = float((diffs**2).mean())
    rmse = float(np.sqrt(mse))
    mae = float(np.abs(diffs).mean())
    return rmse, mae


class Generator(nn.Module):
    def __init__(self, z_dim, sem_dim, feat_dim=2048, hidden=2048):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim + sem_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, feat_dim),
        )
    def forward(self, z, e):
        out = self.net(torch.cat([z, e], dim=1))
        return l2norm(out)   

class Critic(nn.Module):
    def __init__(self, sem_dim, feat_dim=2048, hidden=1024, n_cls=None):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Linear(feat_dim + sem_dim, hidden),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.critic = nn.Linear(hidden, 1)
        self.aux = nn.Linear(hidden, n_cls) if n_cls else None
    def forward(self, x, e):
        h = self.feat(torch.cat([x, e], dim=1))
        s = self.critic(h)
        p = self.aux(h) if self.aux else None
        return s, p

class LinearClassifier(nn.Module):
    def __init__(self, feat_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(feat_dim, num_classes)
    def forward(self, x): return self.fc(x)

# ---------- 梯度惩罚 ----------
def grad_penalty(critic_func, real_x, fake_x, e, gp_lambda=10.0):
    bsz = real_x.size(0)
    eps = torch.rand(bsz, 1, device=real_x.device)
    inter = eps * real_x + (1 - eps) * fake_x
    inter.requires_grad_(True)
    out = critic_func(inter, e)
    grad = torch.autograd.grad(
        outputs=out,
        inputs=inter,
        grad_outputs=torch.ones_like(out),
        create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    grad_norm = grad.view(bsz, -1).norm(2, dim=1)
    return gp_lambda * ((grad_norm - 1.0) ** 2).mean()

# ---------- 训练 seen ----------
def train_fclswgan_on_seen(Xtr, ytr, E, train_classes,
                           lambda_cls=1.0, z_dim=100, n_critic=5, gp_lambda=10.0,
                           lr_g=1e-4, lr_d=4e-4, batch_size=256, epochs=30,
                           log_tag=""):
    feat_dim = Xtr.size(1); sem_dim = E.size(1)
    mask = torch.tensor([int(int(t.item()) in set(train_classes)) for t in ytr],
                        device=ytr.device, dtype=torch.bool)
    Xs, Ys = Xtr[mask], ytr[mask]
    loader = DataLoader(TensorDataset(Xs, Ys), batch_size=batch_size, shuffle=True, drop_last=True)

    G = Generator(z_dim, sem_dim, feat_dim).to(device)
    D = Critic(sem_dim, feat_dim, n_cls=len(train_classes)).to(device)

    optG = optim.Adam(G.parameters(), lr=lr_g, betas=(0.5, 0.9))
    optD = optim.Adam(D.parameters(), lr=lr_d, betas=(0.5, 0.9))
    ce   = nn.CrossEntropyLoss(label_smoothing=0.05)
    cls2idx = {c:i for i,c in enumerate(sorted(train_classes))}

    for ep in range(epochs):
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            eb = E[yb]; bsz = xb.size(0)
            y_idx = torch.tensor([cls2idx[int(t.item())] for t in yb], device=device)

            # ----- Train D -----
            for _ in range(n_critic):
                z = torch.randn(bsz, z_dim, device=device)
                xf = G(z, eb).detach()
                sr, pr = D(xb, eb)
                sf, pf = D(xf, eb)
                loss_D = -(sr.mean() - sf.mean()) \
                         + grad_penalty(lambda a,b: D(a,b)[0], xb, xf, eb, gp_lambda) \
                         + lambda_cls * ce(pr, y_idx)
                optD.zero_grad(); loss_D.backward(); optD.step()

            # ----- Train G -----
            z = torch.randn(bsz, z_dim, device=device)
            xf = G(z, eb)
            sf, pf = D(xf, eb)
            loss_G = -sf.mean() + lambda_cls * ce(pf, y_idx)
            optG.zero_grad(); loss_G.backward(); optG.step()

        if (ep+1) % 5 == 0 or ep == 0:
            print(f"    [G/D]{log_tag} epoch {ep+1}/{epochs} done")
    return G, None

# ---------- 合成未见类特征 ----------
@torch.inference_mode()
def synthesize_features(G, E, class_ids, n_gen=400, z_dim=100):
    feats, labels = [], []
    for c in class_ids:
        e = E[torch.tensor([c], device=device)].repeat(n_gen, 1)
        z = torch.randn(n_gen, z_dim, device=device)
        feats.append(G(z, e))
        labels += [c]*n_gen
    return torch.cat(feats, 0), torch.tensor(labels, device=device)

# ---------- 线性分类器 ----------
def train_linear_classifier(X, y, class_ids, lr=1e-4, epochs=30, bs=512):
    cls2idx = {c:i for i,c in enumerate(sorted(class_ids))}
    y_idx = torch.tensor([cls2idx[int(t.item())] for t in y], device=device)
    loader = DataLoader(TensorDataset(X, y_idx), batch_size=bs, shuffle=True)
    C = LinearClassifier(X.size(1), len(class_ids)).to(device)
    opt = optim.Adam(C.parameters(), lr=lr)
    ce = nn.CrossEntropyLoss(label_smoothing=0.05)
    for ep in range(epochs):
        for xb, yb in loader:
            loss = ce(C(xb), yb)
            opt.zero_grad(); loss.backward(); opt.step()
        if (ep+1) % 10 == 0 or ep == 0:
            print(f"    [CLS] epoch {ep+1}/{epochs} done")
    return C, cls2idx

@torch.inference_mode()
def predict_with_linear(C, X, class_ids, cls2idx):
    logits = C(X)
    pred_idx = logits.argmax(1).cpu().numpy()
    idx2cls = {v:k for k,v in cls2idx.items()}
    return np.array([idx2cls[int(i)] for i in pred_idx], dtype=np.int64)

# ---------- 内层 3折调 λ ----------
def choose_lambda_inner_3fold(Xtr_full, ytr_full, E, lam_grid=(0.5,1.0,2.0),
                              z_dim=100, n_gen=300, epochs_gan=15):
    seen_classes = unique_sorted_cpu(ytr_full)
    folds = [seen_classes[0:15], seen_classes[15:30], seen_classes[30:45]]
    print(f"[CV] Start inner 3-fold; λ candidates = {list(lam_grid)}")
    best_mean, best_lam = -1.0, None
    for lam in lam_grid:
        fold_accs = []
        for fi, val_classes in enumerate(folds, start=1):
            train_classes = [c for c in seen_classes if c not in set(val_classes)]
            G, _ = train_fclswgan_on_seen(Xtr_full, ytr_full, E, train_classes,
                                          lambda_cls=lam, epochs=epochs_gan,
                                          log_tag=f"(λ={lam},fold={fi})")
            Xg_val, yg_val = synthesize_features(G, E, val_classes, n_gen=n_gen)
            C_val, cls2idx_val = train_linear_classifier(Xg_val, yg_val, val_classes, epochs=10)
            _, _, Xva, yva = split_by_val_classes(Xtr_full, ytr_full, val_classes)
            y_pred = predict_with_linear(C_val, Xva, val_classes, cls2idx_val)
            acc = accuracy_manual(yva.cpu().numpy().tolist(), y_pred.tolist())
            fold_accs.append(acc)
        mean_acc = float(np.mean(fold_accs))
        print(f"[CV] λ={lam} | mean acc = {mean_acc:.4f}")
        if mean_acc > best_mean:
            best_mean, best_lam = mean_acc, lam
    print(f"[CV] Best λ = {best_lam}")
    return best_lam

# ---------- 外层 10折 ----------
def run_outer_10fold_fclswgan(emb, lam_grid=(0.5,1.0,2.0),
                              z_dim=100, n_gen=400,
                              epochs_gan=30, epochs_cls=30,
                              folds_to_run=10):
    print("[DEBUG] Enter run_outer_10fold_fclswgan")
    E_all = l2norm(emb.float().to(device))
    results = []

    for i in range(1, folds_to_run+1):
        print(f"\n========== Outer Fold {i:02d} ==========")
        Xtr = l2norm(globals()[f"X_train_{i}"].float().to(device))
        ytr = globals()[f"y_train_{i}"].long().to(device)
        Xte = l2norm(globals()[f"X_test_{i}"].float().to(device))
        yte = globals()[f"y_test_{i}"].long().to(device)

        best_lam = choose_lambda_inner_3fold(
            Xtr, ytr, E_all, lam_grid, z_dim, n_gen, epochs_gan=max(8, epochs_gan//3)
        )
        print(f"Chosen λ (inner 3-fold): {best_lam}")

        seen_classes = unique_sorted_cpu(ytr)
        unseen_ids = unique_sorted_cpu(yte)
        G, _ = train_fclswgan_on_seen(Xtr, ytr, E_all, seen_classes,
                                      lambda_cls=best_lam, z_dim=z_dim, epochs=epochs_gan,
                                      log_tag="(outer)")

        Xg, yg = synthesize_features(G, E_all, unseen_ids, n_gen=n_gen, z_dim=z_dim)
        C, cls2idx = train_linear_classifier(Xg, yg, unseen_ids, epochs=epochs_cls)

        y_pred = predict_with_linear(C, Xte, unseen_ids, cls2idx)
        y_true = yte.cpu().numpy().tolist()
        acc = accuracy_manual(y_true, y_pred.tolist())
        cm  = confusion_matrix_manual(y_true, y_pred.tolist(), unseen_ids)
        p_macro, r_macro, f1_macro = macro_prf_from_cm(cm)

        with torch.no_grad():
            gen_means = {}
            for c in unseen_ids:
                e = E_all[torch.tensor([c], device=device)]
                z = torch.randn(n_gen, z_dim, device=device)
                mu_c = G(z, e.repeat(n_gen, 1)).mean(dim=0)
                gen_means[int(c)] = mu_c
            rmse, mae = rmse_mae_visual(Xte, y_pred.tolist(), gen_means)

        print(f"Acc: {acc:.4f} | Macro P/R/F1: {p_macro:.4f}/{r_macro:.4f}/{f1_macro:.4f} | RMSE {rmse:.4f} MAE {mae:.4f}")
        results.append([best_lam, acc, p_macro, r_macro, f1_macro, rmse, mae])

    arr = np.array(results, dtype=float)
    mean_vals, std_vals = arr[:,1:].mean(0), arr[:,1:].std(0)
    print("\n========== 10-Fold Summary ==========")
    print(f"Accuracy : {mean_vals[0]:.4f} ± {std_vals[0]:.4f}")
    print(f"Macro P  : {mean_vals[1]:.4f} ± {std_vals[1]:.4f}")
    print(f"Macro R  : {mean_vals[2]:.4f} ± {std_vals[2]:.4f}")
    print(f"Macro F1 : {mean_vals[3]:.4f} ± {std_vals[3]:.4f}")
    print(f"RMSE     : {mean_vals[4]:.6f} ± {std_vals[4]:.6f}")
    print(f"MAE      : {mean_vals[5]:.6f} ± {std_vals[5]:.6f}")

# ---------- 启动运行 ----------
run_outer_10fold_fclswgan(
    emb,
    lam_grid=(0.5, 1.0, 2.0),
    z_dim=100, n_gen=400,
    epochs_gan=35, epochs_cls=35,
    folds_to_run=10
)


>>> Script loaded. Device: cuda
[DEBUG] Enter run_outer_10fold_fclswgan

========== Outer Fold 01 ==========
[CV] Start inner 3-fold; λ candidates = [0.5, 1.0, 2.0]
    [G/D](λ=0.5,fold=1) epoch 1/11 done
    [G/D](λ=0.5,fold=1) epoch 5/11 done
    [G/D](λ=0.5,fold=1) epoch 10/11 done
    [CLS] epoch 1/10 done
    [CLS] epoch 10/10 done
    [G/D](λ=0.5,fold=2) epoch 1/11 done
    [G/D](λ=0.5,fold=2) epoch 5/11 done
    [G/D](λ=0.5,fold=2) epoch 10/11 done
    [CLS] epoch 1/10 done
    [CLS] epoch 10/10 done
    [G/D](λ=0.5,fold=3) epoch 1/11 done
    [G/D](λ=0.5,fold=3) epoch 5/11 done
    [G/D](λ=0.5,fold=3) epoch 10/11 done
    [CLS] epoch 1/10 done
    [CLS] epoch 10/10 done
[CV] λ=0.5 | mean acc = 0.1909
    [G/D](λ=1.0,fold=1) epoch 1/11 done
    [G/D](λ=1.0,fold=1) epoch 5/11 done
    [G/D](λ=1.0,fold=1) epoch 10/11 done
    [CLS] epoch 1/10 done
    [CLS] epoch 10/10 done
    [G/D](λ=1.0,fold=2) epoch 1/11 done
    [G/D](λ=1.0,fold=2) epoch 5/11 done
    [G/D](λ=1.0,fold=2) epoc

In [25]:
# ===== F-CLSWGAN · 10-fold CV (与单折0.8+版一致风格) =====
import numpy as np
import torch, torch.nn as nn, torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# ---------- utils ----------
def l2n(x): 
    return x / (x.norm(dim=1, keepdim=True) + 1e-12)

def macro_from_cm(cm_torch):
    cm = cm_torch.to(torch.float64)
    tp = cm.diag()
    fp = cm.sum(0) - tp
    fn = cm.sum(1) - tp
    prec_i = torch.where(tp+fp>0, tp/(tp+fp), torch.zeros_like(tp))
    reca_i = torch.where(tp+fn>0, tp/(tp+fn), torch.zeros_like(tp))
    f1_i   = torch.where(prec_i+reca_i>0, 2*prec_i*reca_i/(prec_i+reca_i), torch.zeros_like(tp))
    return float(prec_i.mean()), float(reca_i.mean()), float(f1_i.mean())

def confusion_manual(y_true_list, y_pred_list, labels_sorted):
    L = len(labels_sorted); idx = {c:i for i,c in enumerate(labels_sorted)}
    cm = torch.zeros((L, L), dtype=torch.int64)
    for t,p in zip(y_true_list, y_pred_list):
        cm[idx[t], idx[p]] += 1
    return cm

def rmse_mae_visual(X_real, y_pred, gen_means_dict):
    diffs = []
    for xr, c in zip(X_real, y_pred):
        mu = gen_means_dict[int(c)]
        diffs.append((xr - mu).detach().cpu().numpy())
    diffs = np.stack(diffs, 0)
    mse = float((diffs**2).mean())
    return float(np.sqrt(mse)), float(np.abs(diffs).mean())

# ---------- models ----------
class Generator(nn.Module):
    def __init__(self, z_dim, sem_dim, feat_dim=2048):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim + sem_dim, 1024), nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1024),           nn.LeakyReLU(0.2, True),
            nn.Linear(1024, feat_dim),
        )
    def forward(self, z, e):
        return l2n(self.net(torch.cat([z, e], 1)))

class Discriminator(nn.Module):
    def __init__(self, sem_dim, feat_dim=2048, n_cls=10):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Linear(feat_dim + sem_dim, 1024), nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 512),                nn.LeakyReLU(0.2, True),
        )
        self.critic = nn.Linear(512, 1)
        self.aux    = nn.Linear(512, n_cls)
    def forward(self, x, e):
        h = self.feat(torch.cat([x, e], 1))
        return self.critic(h), self.aux(h)

# ---------- training pieces ----------
def gp_term(D, real_x, fake_x, e_txt, lambda_gp=10.0):
    b = real_x.size(0)
    eps = torch.rand(b, 1, device=real_x.device)
    inter = eps*real_x + (1-eps)*fake_x
    inter.requires_grad_(True)
    score, _ = D(inter, e_txt)
    grad = torch.autograd.grad(outputs=score, inputs=inter,
                               grad_outputs=torch.ones_like(score),
                               create_graph=True, retain_graph=True, only_inputs=True)[0]
    return lambda_gp * ((grad.view(b, -1).norm(2, dim=1) - 1)**2).mean()

def train_seen_fclswgan(X_seen, y_seen, E_all, seen_ids,
                        Z_dim=128, batch_size=256, epochs=30,
                        n_critic=5, lambda_gp=10.0, lambda_aux=1.0):
    D_img = X_seen.size(1); D_txt = E_all.size(1)
    G = Generator(Z_dim, D_txt, D_img).to(device)
    D = Discriminator(D_txt, D_img, n_cls=len(seen_ids)).to(device)

    optD = optim.Adam(D.parameters(), lr=1e-4, betas=(0.5, 0.9))
    optG = optim.Adam(G.parameters(), lr=1e-4, betas=(0.5, 0.9))
    ce   = nn.CrossEntropyLoss()

    # 均匀采样每类
    idx_by_cls = {c: torch.where(y_seen==c)[0] for c in seen_ids}
    def sample_seen_batch(B):
        k = len(seen_ids); per = max(1, B//k)
        xs, ys, es = [], [], []
        for c in seen_ids:
            idxs = idx_by_cls[c]
            take = idxs[torch.randint(0, idxs.numel(), (per,), device=device)]
            xs.append(X_seen[take])
            ys.append(torch.full((per,), seen_ids.index(c), device=device))
            es.append(E_all[c].unsqueeze(0).expand(per, -1))
        Xb = torch.cat(xs, 0)[:B]
        yb = torch.cat(ys, 0)[:B]
        eb = torch.cat(es, 0)[:B]
        return Xb, yb, eb

    for _ in range(epochs):
        for _ in range(max(200, len(seen_ids))):
            # 更新 D
            for _ in range(n_critic):
                Xr, yr_idx, er = sample_seen_batch(batch_size)
                z  = torch.randn(Xr.size(0), Z_dim, device=device)
                xf = G(z, er).detach()
                sr, pr = D(Xr, er)
                sf, pf = D(xf, er)
                loss_D = -(sr.mean() - sf.mean()) + gp_term(D, Xr, xf, er, lambda_gp) + lambda_aux*ce(pr, yr_idx)
                optD.zero_grad(); loss_D.backward(); optD.step()
            # 更新 G
            Xr, yr_idx, er = sample_seen_batch(batch_size)
            z  = torch.randn(Xr.size(0), Z_dim, device=device)
            xf = G(z, er)
            sf, pf = D(xf, er)
            loss_G = -sf.mean() + lambda_aux*ce(pf, yr_idx)
            optG.zero_grad(); loss_G.backward(); optG.step()
    return G

@torch.inference_mode()
def synthesize_unseen(G, E_all, unseen_ids, per_cls=400, Z_dim=128):
    feats, labs = [], []
    for j, cid in enumerate(unseen_ids):
        e = E_all[cid].unsqueeze(0).expand(per_cls, -1)
        z = torch.randn(per_cls, Z_dim, device=device)
        feats.append(G(z, e))
        labs.append(torch.full((per_cls,), j, device=device))
    return torch.cat(feats,0), torch.cat(labs,0)

def train_linear_cls(X_syn, y_syn, n_cls, steps=200, bs=512, lr=5e-3, wd=1e-4):
    clf = nn.Linear(X_syn.size(1), n_cls).to(device)
    opt = optim.AdamW(clf.parameters(), lr=lr, weight_decay=wd)
    ce  = nn.CrossEntropyLoss()
    for _ in range(steps):
        idx = torch.randint(0, X_syn.size(0), (bs,), device=device)
        xb, yb = X_syn[idx], y_syn[idx]
        loss = ce(clf(xb), yb)
        opt.zero_grad(); loss.backward(); opt.step()
    return clf

# ---------- 10-fold ----------
def run_10fold_fclswgan(emb,
                        Z_dim=128, batch_size=256, epochs=30,
                        n_critic=5, lambda_gp=10.0, lambda_aux=1.0,
                        gen_per_cls=400, cls_steps=200, cls_bs=512, cls_lr=5e-3, cls_wd=1e-4):
    E_all = l2n(emb.float().to(device))
    results = []  # [acc, P, R, F1, RMSE, MAE]

    for i in range(1, 11):
        print(f"\n========== Fold {i:02d} ==========")
        Xtr = l2n(globals()[f"X_train_{i}"].float().to(device))
        ytr = globals()[f"y_train_{i}"].long().to(device)
        Xte = l2n(globals()[f"X_test_{i}"].float().to(device))
        yte = globals()[f"y_test_{i}"].long().to(device)

        seen_ids   = sorted(set(ytr.cpu().numpy().tolist()))
        unseen_ids = sorted(set(yte.cpu().numpy().tolist()))

        # 1) 训练 GAN（仅 seen）
        G = train_seen_fclswgan(Xtr, ytr, E_all, seen_ids,
                                Z_dim, batch_size, epochs, n_critic, lambda_gp, lambda_aux)

        # 2) 合成未见 + 训练线性分类器
        X_syn, y_syn = synthesize_unseen(G, E_all, unseen_ids, per_cls=gen_per_cls, Z_dim=Z_dim)
        clf = train_linear_cls(X_syn, y_syn, n_cls=len(unseen_ids),
                               steps=cls_steps, bs=cls_bs, lr=cls_lr, wd=cls_wd)

        # 3) 测试集评估
        with torch.no_grad():
            logits = clf(Xte)
            pred_small = logits.argmax(1).cpu().numpy()
            y_pred = np.array([unseen_ids[int(j)] for j in pred_small], dtype=np.int64)
        y_true = yte.cpu().numpy().tolist()
        acc = float((np.array(y_true) == y_pred).mean())
        cm  = confusion_manual(y_true, y_pred.tolist(), unseen_ids)
        P, R, F1 = macro_from_cm(cm)

        # 4) RMSE / MAE（用每个未见类的生成均值）
        with torch.no_grad():
            gen_means = {}
            for c in unseen_ids:
                e = E_all[torch.tensor([c], device=device)]
                z = torch.randn(gen_per_cls, Z_dim, device=device)
                mu = G(z, e.repeat(gen_per_cls,1)).mean(0)
                gen_means[int(c)] = mu
            rmse, mae = rmse_mae_visual(Xte, y_pred.tolist(), gen_means)

        print(f"Acc={acc:.4f}  MacroP={P:.4f}  MacroR={R:.4f}  F1={F1:.4f}  RMSE={rmse:.6f}  MAE={mae:.6f}")
        results.append([acc, P, R, F1, rmse, mae])

    M = np.array(results, dtype=float)
    mean, std = M.mean(0), M.std(0)
    print("\n========== 10-Fold Summary ==========")
    print(f"Accuracy : {mean[0]:.4f} ± {std[0]:.4f}")
    print(f"Macro P  : {mean[1]:.4f} ± {std[1]:.4f}")
    print(f"Macro R  : {mean[2]:.4f} ± {std[2]:.4f}")
    print(f"Macro F1 : {mean[3]:.4f} ± {std[3]:.4f}")
    print(f"RMSE     : {mean[4]:.6f} ± {std[4]:.6f}")
    print(f"MAE      : {mean[5]:.6f} ± {std[5]:.6f}")
    return mean, std, results

# ================== 直接开跑 ==================
# 需已定义: X_train_1..10 / y_train_1..10 / X_test_1..10 / y_test_1..10 / emb
run_10fold_fclswgan(
    emb,
    Z_dim=128, batch_size=256, epochs=30,
    n_critic=5, lambda_gp=10.0, lambda_aux=1.0,
    gen_per_cls=500, cls_steps=400, cls_bs=512, cls_lr=5e-3, cls_wd=1e-4
)



========== Fold 01 ==========
Acc=0.8149  MacroP=0.7576  MacroR=0.8074  F1=0.7750  RMSE=0.023533  MAE=0.010660

========== Fold 02 ==========
Acc=0.7377  MacroP=0.7274  MacroR=0.7023  F1=0.7104  RMSE=0.022102  MAE=0.010902

========== Fold 03 ==========
Acc=0.6549  MacroP=0.5528  MacroR=0.5992  F1=0.5476  RMSE=0.022990  MAE=0.010459

========== Fold 04 ==========
Acc=0.5690  MacroP=0.6011  MacroR=0.5818  F1=0.5276  RMSE=0.022821  MAE=0.010553

========== Fold 05 ==========
Acc=0.6300  MacroP=0.5290  MacroR=0.5362  F1=0.5034  RMSE=0.022713  MAE=0.010834

========== Fold 06 ==========
Acc=0.5554  MacroP=0.5987  MacroR=0.5507  F1=0.4787  RMSE=0.023298  MAE=0.010497

========== Fold 07 ==========
Acc=0.7499  MacroP=0.7426  MacroR=0.7427  F1=0.6802  RMSE=0.022671  MAE=0.010222

========== Fold 08 ==========
Acc=0.5676  MacroP=0.5246  MacroR=0.6282  F1=0.5447  RMSE=0.022519  MAE=0.010659

========== Fold 09 ==========
Acc=0.5238  MacroP=0.6725  MacroR=0.5260  F1=0.5125  RMSE=0.022825  MAE=0

(array([0.62792809, 0.6175357 , 0.62402268, 0.57458616, 0.02294955,
        0.01054261]),
 array([0.10434926, 0.09680419, 0.09074496, 0.10174217, 0.00051851,
        0.00025861]),
 [[0.814921681780709,
   0.75764654811634,
   0.8074137187874489,
   0.7749533003384872,
   0.02353292648547946,
   0.01065992470830679],
  [0.7377085162423178,
   0.7273761471985736,
   0.7023065109898747,
   0.7104083942275918,
   0.02210178446927936,
   0.010901729576289654],
  [0.6549218233869192,
   0.5528043343796271,
   0.5991558113753115,
   0.5476404489906702,
   0.022989596446342443,
   0.010458569042384624],
  [0.569004758948893,
   0.6010939560331613,
   0.5818430138187344,
   0.5275631282625779,
   0.022820804601823622,
   0.010553264990448952],
  [0.6299902629016553,
   0.5290425804896919,
   0.5361627516444974,
   0.5034188959160839,
   0.022713455838813804,
   0.010834131389856339],
  [0.5553519061583577,
   0.5987233573027096,
   0.5507301966633419,
   0.47869022804628064,
   0.02329770133190